In [ ]:
import pandas as pd
import os
import re
from tqdm.auto import tqdm
import shutil


In [ ]:
# 각 기업에서 발생한 라벨 1의 개수를 추적하는 코드
data = pd.read_csv('/content/drive/MyDrive/Kwargs/020. 전처리/A0_Sports_Obituary_filterbase.csv')

# 2. 'full_text' 컬럼이 비어 있는 경우 'content' 컬럼으로 대체
# pandas의 .fillna()를 사용하여 'full_text'가 비어있으면 'content' 컬럼의 값으로 채움
data['full_text'] = data['full_text'].fillna(data['content'])
data['full_text'] = data['content'].fillna(data['full_text'])

# 3. 'full_text'가 비어있는 경우뿐만 아니라 값이 있지만 공백인 경우도 대체
# 공백이나 빈 문자열이 포함된 경우에도 'content' 값으로 대체
data['full_text'] = data.apply(lambda row: row['content'] if pd.isna(row['full_text']) or row['full_text'].strip() == "" else row['full_text'], axis=1)
data['content'] = data.apply(lambda row: row['full_text'] if pd.isna(row['content']) or row['content'].strip() == "" else row['content'], axis=1)

# 결과 확인
print(data[['full_text', 'content']].head())



                                           full_text  \
0  “주문량 넘쳐 규모 7배이상 확대”유럽시장 공략에 본격 투자 나서 LG화학-삼성SD...   
1  머니투데이 고석용 기자 사람인 구직자 대상 조사 2위에 카카오, 5위 네이버구직자들...   
2  입사 선호기업 상위권에 IT·벤처기업들이 도약하는 양상이다.구인구직 매칭플랫폼 사람...   
3   '코스피 2196·코스닥 738' 코스피 지수가 2200선 안착에 또다시 실패했다...   
4  -올해 주총 집중도 지난해 대비 완화-26일과 27일 주총 집중예상일 추가 지정 검...   

                                             content  
0  “주문량 넘쳐 규모 7배이상 확대”유럽시장 공략에 본격 투자 나서 LG화학-삼성SD...  
1  머니투데이 고석용 기자 사람인 구직자 대상 조사 2위에 카카오, 5위 네이버구직자들...  
2  입사 선호기업 상위권에 IT·벤처기업들이 도약하는 양상이다.구인구직 매칭플랫폼 사람...  
3   '코스피 2196·코스닥 738' 코스피 지수가 2200선 안착에 또다시 실패했다...  
4  -올해 주총 집중도 지난해 대비 완화-26일과 27일 주총 집중예상일 추가 지정 검...  


In [ ]:
# 라벨 1의 데이터 추출 및 각 기업별로 라벨 1의 개수 세기
data['labelso'].value_counts()

,count
labelso,
0,169559
1,12332


In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
import pandas as pd
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch


# 데이터가 'full_text'와 'labelso' 컬럼을 포함해야 함
# 만약 그렇지 않다면 적절한 전처리가 필요함
data = data[['full_text', 'labelso']]

In [ ]:
import pandas as pd
from sklearn.utils import resample

# 라벨 1과 라벨 0을 분리
label_1 = data[data['labelso'] == 1]
label_0 = data[data['labelso'] == 0]

# 라벨 1의 수에 맞춰 라벨 0을 랜덤하게 다운샘플링
label_0_downsampled = resample(label_0,
                               replace=False,  # 복원 없이 샘플링
                               n_samples=len(label_1),  # 라벨 1과 같은 수로 샘플링
                               random_state=42)  # 재현성을 위한 random state 설정

# 다운샘플된 라벨 0과 라벨 1을 다시 결합
data_balanced = pd.concat([label_1, label_0_downsampled])

# 데이터 섞기
data_balanced = data_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# 결과 출력
print(data_balanced['labelso'].value_counts())


labelso
1    12332
0    12332
Name: count, dtype: int64


In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# 라벨 컬럼명을 'label'로 변경
data = data.rename(columns={'labelso': 'label'})

# 이후 프로세스에서 'label'을 사용하여 처리
label_1 = data[data['label'] == 1]
label_0 = data[data['label'] == 0]

# 동일한 샘플링 및 파이프라인 유지
label_1_sampled = resample(label_1, n_samples=12332, random_state=42)
label_0_sampled = resample(label_0, n_samples=12332, random_state=42)

data_balanced = pd.concat([label_1_sampled, label_0_sampled])
data_balanced = data_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    data_balanced['full_text'].tolist(),
    data_balanced['label'].tolist(),  # 'label'을 사용
    test_size=0.2,
    random_state=42
)


In [ ]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

# Koelectra 모델과 토크나이저 로드
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)

# 데이터 토크나이징 (max_length = 256)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)# 평균 토큰 길이 계산 = 775로 확인함
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Koelectra 모델과 토크나이저 로드
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)

# 데이터 토크나이징 (max_length = 256)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

# 모든 모델 파라미터를 연속적으로 변환하여 ValueError 방지
for param in model.parameters():
    param.data = param.data.contiguous()

# 모델 평가를 위한 지표 정의
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Kwargs/020. 전처리/A0_Sports_Obituary_koelectra',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    eval_strategy="epoch",  # deprecated 된 evaluation_strategy 대신 eval_strategy 사용
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=2,
    load_best_model_at_end=True,
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # 평가 지표 추가
)

# 모델 학습
trainer.train()

# 학습 완료 후 모델 저장
model.save_pretrained('/content/drive/MyDrive/Kwargs/020. 전처리/A0_Sports_Obituary_koelectra')
tokenizer.save_pretrained('/content/drive/MyDrive/Kwargs/020. 전처리/A0_Sports_Obituary_koelectra')


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.669900,0.673256,0.620000,0.687500,0.440000,0.536585
2,0.625200,0.616291,0.660000,0.880952,0.370000,0.521127
3,0.605000,0.562249,0.690000,0.865385,0.450000,0.592105


('/content/drive/MyDrive/Kwargs/020. 전처리/A0_Sports_Obituary_koelectra/tokenizer_config.json',
 '/content/drive/MyDrive/Kwargs/020. 전처리/A0_Sports_Obituary_koelectra/special_tokens_map.json',
 '/content/drive/MyDrive/Kwargs/020. 전처리/A0_Sports_Obituary_koelectra/vocab.txt',
 '/content/drive/MyDrive/Kwargs/020. 전처리/A0_Sports_Obituary_koelectra/added_tokens.json')

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# 평가 결과 출력
metrics = trainer.evaluate(eval_dataset=val_dataset)
print(f"Accuracy: {metrics['eval_accuracy']}")
print(f"Precision: {metrics['eval_precision']}")
print(f"Recall: {metrics['eval_recall']}")
print(f"F1-score: {metrics['eval_f1']}")


Accuracy: 0.69
Precision: 0.8653846153846154
Recall: 0.45
F1-score: 0.5921052631578947


In [ ]:
# 테스트를 위한 텍스트 입력 및 토큰화
test_text = input()
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True, max_length=256)

# 모델을 사용한 예측
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)

# 결과 라벨 출력
label = predictions.item()
print(f"예측된 라벨: {label}")


(엑스포츠뉴스 잠실, 박지영 기자) 5일 오후 서울 송파구 잠실야구장에서 열린 '2024 신한 SOL Bank KBO 포스트시즌' KT 위즈와 LG 트윈스의 준플레이오프 1차전 경기, 5회초 KT 선두타자 황재균이 헛스윙 삼진 아웃으로 물러나고 있다.  박지영 기자 jypark@xportsnews.com
예측된 라벨: 1


In [ ]:
# 테스트를 위한 텍스트 입력 및 토큰화
test_text = input()
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True, max_length=256)

# 모델을 사용한 예측
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)

# 결과 라벨 출력
label = predictions.item()
print(f"예측된 라벨: {label}")

정상 네이버페이 창 결제방식 다양, 피싱 사이트 ‘일반결제와 네이버페이 간편결제’ 두 가지만 서비스 피싱 사이트에서 무통장입금 거래 선택시 사기 계좌로 연결, 송금 시 물품 수령 불가능 누리랩, 네이버페이 피싱 사이트 분석 보고서 발표  [보안뉴스 김영명 기자] 네이버(NAVER)의 결제 시스템인 네이버페이(NAVER PAY)를 이용한 네이버쇼핑은 최근 쿠팡을 넘어 국내 점유율 22%를 기록하며 전자상거래 분야 시장점유율에서 1위를 차지했다. 네이버쇼핑 서비스는 물품의 구매와 결제 등을 매우 편리하게 만들어 주는 시스템이자 서비스다. 하지만 최근 결제 창 등을 사칭해 사용자의 정보 탈취 혹은 사기 행위를 벌이는 피싱 사이트가 다수 발견되어 사용자들의 주의가 필요하다.   ▲네이버페이 피싱 사이트의 최초 구성 화면[자료=누리랩 보안팀]  보안전문기업 누리랩이 발표한 ‘네이버페이 피싱 사이트 분석 보고서’를 살펴보면 올해 8월에 파악된 피싱 사이트의 URL 건수는 중복을 제외하고 전달보다 4만 7,177건이 증가한 8만 5,768건으로 나타났다.  해당 피싱 사이트에 처음으로 접속하면 사기 물품의 간략한 구매 정보와 함께 배송지를 등록하라는 메시지가 출력된다. 네이버쇼핑 구매 물품의 정식 결제 창과 매우 비슷한 사이트 구성을 띄고 있지만, 이를 자세히 살펴보면 여러 가지 차이점이 존재한다.  첫 번째 차이는 바로 사이트의 URL이다. 네이버페이의 공식 결제창 URL은 ‘or***.pay.nav**.com’ 형식이다. 하지만 피싱 사이트의 URL은 ‘naverpay-****.cafe’와 ‘naverpay-page****.cafe’로 표기된다. 또한 네이버쇼핑 공식 사이트의 실제 결제 창에서는 URL의 패스, 파라미터, 프레그먼트 등의 구성 요소에 판매자 상점 정보와 구매 물품 정보 등이 포함돼 있다. 하지만 피싱 사이트의 URL에서는 해당 정보 등을 확인할 수 없다. 따라서 URL 구성 화면부터 해당 사이트는 피싱이나 사기와 같은 악의적 행위를 목적으로 한 사이트임

In [ ]:
# 이제 이 모델로 데이터 필터링해서 1로 분류된 데이터 버리기